In [2]:
import numpy as np
import pandas as pd

shops = pd.read_csv('input/shops.csv', index_col=1)
print(shops.shape)
shops.head()

(60, 1)


,shop_name
shop_id,
0,"!Якутск Орджоникидзе, 56 фран"
1,"!Якутск ТЦ ""Центральный"" фран"
2,"Адыгея ТЦ ""Мега"""
3,"Балашиха ТРК ""Октябрь-Киномир"""
4,"Волжский ТЦ ""Волга Молл"""


In [2]:
categs = pd.read_csv('input/item_categories.csv', index_col=1)
categs.head()

,item_category_name
item_category_id,
0,PC - Гарнитуры/Наушники
1,Аксессуары - PS2
2,Аксессуары - PS3
3,Аксессуары - PS4
4,Аксессуары - PSP


In [3]:
items = pd.read_csv('input/items.csv', index_col=1)
items.head()

,item_name,item_category_id
item_id,,
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40
1,!ABBYY FineReader 12 Professional Edition Full...,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,40
4,***КОРОБКА (СТЕКЛО) D,40


In [4]:
transactions = pd.read_csv('input/sales_train_v2.csv')
print('shape:', transactions.shape)
print('max date_block_num:', transactions['date_block_num'].max())
transactions.head()

shape: (2935849, 6)
max date_block_num: 33


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
test_data = pd.read_csv('input/test.csv', index_col=0)
print(test_data.shape)
test_data.head()

(214200, 2)


,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268


In [6]:
monthly_totals = transactions.groupby(['date_block_num', 'shop_id', 'item_id'],as_index=False)[['item_cnt_day']].sum()
monthly_totals['item_cnt_day'].describe()

count    1.609124e+06
mean     2.267200e+00
std      8.649882e+00
min     -2.200000e+01
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      2.253000e+03
Name: item_cnt_day, dtype: float64

In [8]:
#save file
monthly_totals.to_csv('input/monthly_totals.csv', index=False)

In [9]:
#add category id
monthly_totals = monthly_totals.set_index('item_id').join(items).drop('item_name', axis=1).reset_index()

In [10]:
monthly_totals.to_csv('input/monthly_totals_categ.csv', index=False)
monthly_totals.head()

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id
0,0,20,54,1.0,40
1,1,15,55,2.0,76
2,1,18,55,1.0,76
3,1,19,55,1.0,76
4,1,20,55,1.0,76


In [11]:
#change date_block_num
def get_year_from_date_block_num(date_block_num):
    if date_block_num < 12:
        return 2013
    elif date_block_num < 24:
        return 2014
    else:
        return 2015

def get_month_from_date_block_num(date_block_num):
    return int(date_block_num % 12)

print('date block: 33 =' , get_year_from_date_block_num(33), get_month_from_date_block_num(33))


date block: 33 = 2015 9


In [12]:
def get_year_for_row(row):
    return get_year_from_date_block_num(row['date_block_num'])

monthly_totals['year'] = monthly_totals.apply(lambda row: get_year_for_row(row), axis=1)
monthly_totals.head()

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id,year
0,0,20,54,1.0,40,2014
1,1,15,55,2.0,76,2014
2,1,18,55,1.0,76,2014
3,1,19,55,1.0,76,2014
4,1,20,55,1.0,76,2014


In [13]:
def get_month_for_row(row):
    return get_month_from_date_block_num(row['date_block_num'])

monthly_totals['month'] = monthly_totals.apply(lambda row: get_month_for_row(row), axis=1)
monthly_totals.head()

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id,year,month
0,0,20,54,1.0,40,2014,8
1,1,15,55,2.0,76,2014,3
2,1,18,55,1.0,76,2014,6
3,1,19,55,1.0,76,2014,7
4,1,20,55,1.0,76,2014,8


In [2]:
#monthly_totals.to_csv('input/monthly_totals_categ_date.csv', index=False)
monthly_totals = pd.read_csv('input/monthly_totals_categ_date.csv')
monthly_totals.head()

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id,year,month
0,0,20,54,1.0,40,2014,8
1,1,15,55,2.0,76,2014,3
2,1,18,55,1.0,76,2014,6
3,1,19,55,1.0,76,2014,7
4,1,20,55,1.0,76,2014,8


In [15]:
#monthly_totals.reset_index()
#monthly_totals.set_index(['date_block_num','shop_id','item_id'], inplace=True)
indexed_monthly_totals = monthly_totals.set_index(['date_block_num','shop_id','item_id'])
indexed_monthly_totals.head()

,,,item_cnt_day,item_category_id,year,month
date_block_num,shop_id,item_id,,,,
20,54,0,1.0,40,2014,8
15,55,1,2.0,76,2014,3
18,55,1,1.0,76,2014,6
19,55,1,1.0,76,2014,7
20,55,1,1.0,76,2014,8


In [8]:
#divide into smaller files
#monthly_totals_0 = monthly_totals.loc[(monthly_totals['shop_id'] >= 0) & (monthly_totals['shop_id'] < 10)]

num_shops = int(monthly_totals['shop_id'].max())
for x in range(num_shops):
    df = monthly_totals.loc[(monthly_totals['shop_id'] == x)]
    print(df.shape)
    df.to_csv('staging/monthly_totals_' + str(x), index=False)
    

(4821, 7)
(3282, 7)
(15553, 7)
(16827, 7)
(23741, 7)
(24108, 7)
(47250, 7)
(32700, 7)
(2465, 7)
(1591, 7)
(14917, 7)
(371, 7)
(20742, 7)
(12105, 7)
(22853, 7)
(31784, 7)
(31303, 7)
(15125, 7)
(30594, 7)
(36630, 7)
(795, 7)
(36830, 7)
(24720, 7)
(4427, 7)
(29141, 7)
(87977, 7)
(32386, 7)
(48438, 7)
(64639, 7)
(27544, 7)
(30980, 7)
(99241, 7)
(5525, 7)
(3544, 7)
(4135, 7)
(34187, 7)
(258, 7)
(25699, 7)
(26109, 7)
(9781, 7)
(2235, 7)
(25316, 7)
(56861, 7)
(21720, 7)
(27085, 7)
(23156, 7)
(36659, 7)
(32033, 7)
(13580, 7)
(11120, 7)
(36203, 7)
(29890, 7)
(27855, 7)
(30166, 7)
(70471, 7)
(12766, 7)
(43876, 7)
(59972, 7)
(37954, 7)


In [16]:
def get_total_for_prev_month(df, date_block_num, shop_id, item_id, prev_month_index):
    target_date_block_num = date_block_num + prev_month_index
    if target_date_block_num < 0:
        return np.nan
    elif target_date_block_num > 33:
        return np.nan
    
    #found = df.loc[(df['date_block_num'] == target_date_block_num) & (df['shop_id'] == shop_id) & (df['item_id'] == item_id)]
    #if found.empty:
    #    return 0
    #else:
    #    return found['item_cnt_day'].iloc[0]
    idx = (int(target_date_block_num), int(shop_id), int(item_id)) 
    if df.index.isin([idx]).any():
        return df.loc[idx]['item_cnt_day']
    else:
        return 0
    
def get_prev_month_total_for_row(row, df, prev_month_index):
    return get_total_for_prev_month(df, row['date_block_num'],row['shop_id'],row['item_id'],prev_month_index)


In [17]:
for x in range(num_shops):
    df = pd.read_csv('staging/monthly_totals_' + str(x))
    idf = df.set_index(['date_block_num','shop_id','item_id'])
    df2 = pd.read_csv('staging/monthly_totals_' + str(x))
    df2['item_cnt_tm1'] = df2.apply(lambda row: get_prev_month_total_for_row(row, idf, -1), axis=1)
    df2['item_cnt_tm2'] = df2.apply(lambda row: get_prev_month_total_for_row(row, idf, -2), axis=1)
    df2['item_cnt_tm12'] = df2.apply(lambda row: get_prev_month_total_for_row(row, idf, -12), axis=1)
    df2['item_cnt_tp1'] = df2.apply(lambda row: get_prev_month_total_for_row(row, idf, 1), axis=1)
    df2.to_csv('staging/monthly_totals_prevs_' + str(x) + '.csv', index=False)
    print(df2.shape)
#training_data.loc[training_data['item_cnt_tp1'] > 0.0]

(4821, 11)
(3282, 11)
(15553, 11)
(16827, 11)
(23741, 11)
(24108, 11)
(47250, 11)
(32700, 11)
(2465, 11)
(1591, 11)
(14917, 11)
(371, 11)
(20742, 11)
(12105, 11)
(22853, 11)
(31784, 11)
(31303, 11)
(15125, 11)
(30594, 11)
(36630, 11)
(795, 11)
(36830, 11)
(24720, 11)
(4427, 11)
(29141, 11)
(87977, 11)
(32386, 11)
(48438, 11)
(64639, 11)
(27544, 11)
(30980, 11)
(99241, 11)
(5525, 11)
(3544, 11)
(4135, 11)
(34187, 11)
(258, 11)
(25699, 11)
(26109, 11)
(9781, 11)
(2235, 11)
(25316, 11)
(56861, 11)
(21720, 11)
(27085, 11)
(23156, 11)
(36659, 11)
(32033, 11)
(13580, 11)
(11120, 11)
(36203, 11)
(29890, 11)
(27855, 11)
(30166, 11)
(70471, 11)
(12766, 11)
(43876, 11)
(59972, 11)
(37954, 11)


In [18]:
dfs = []
for x in range(num_shops):
    df = pd.read_csv('staging/monthly_totals_prevs_' + str(x) + '.csv')
    dfs.append(df)
    
monthly_totals_all = pd.concat(dfs, ignore_index=True)
monthly_totals_all.to_csv('staging/monthly_totals_prevs_all.csv', index=False)

In [23]:
monthly_totals_all.head()

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id,year,month,item_cnt_tm1,item_cnt_tm2,item_cnt_tm12,item_cnt_tp1
8104,27,17,2,1.0,19,2014,5,0.0,0.0,0.0,0.0
8107,30,15,2,1.0,40,2014,3,0.0,0.0,0.0,1.0
8108,30,16,2,1.0,40,2014,4,1.0,0.0,0.0,0.0
8112,31,16,2,1.0,37,2014,4,0.0,0.0,0.0,0.0
8114,32,12,2,1.0,40,2014,0,0.0,0.0,0.0,0.0


In [20]:
#drop rows with nan
monthly_totals_all.dropna(inplace=True)
print(monthly_totals_all.shape)

(876327, 11)


In [25]:
monthly_totals_all.to_csv('input/training_data.csv', index=False)

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id,year,month,item_cnt_tm1,item_cnt_tm2,item_cnt_tm12,item_cnt_tp1
8104,27,17,2,1.0,19,2014,5,0.0,0.0,0.0,0.0
8107,30,15,2,1.0,40,2014,3,0.0,0.0,0.0,1.0
8108,30,16,2,1.0,40,2014,4,1.0,0.0,0.0,0.0
8112,31,16,2,1.0,37,2014,4,0.0,0.0,0.0,0.0
8114,32,12,2,1.0,40,2014,0,0.0,0.0,0.0,0.0
